In [1]:
import os, time
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from torchsummary import summary
from tqdm import tqdm
device = "cuda"

# Clean broken image

In [ ]:
from sklearn.utils import shuffle
df = pd.read_csv("./dataset/train.csv")
broken_list = []
broken_list.append(df.loc[df['ImageId'] == 'ID_1a5a10365'].index)
broken_list.append(df.loc[df['ImageId'] == 'ID_4d238ae90'].index)
broken_list.append(df.loc[df['ImageId'] == 'ID_408f58e9f'].index)
broken_list.append(df.loc[df['ImageId'] == 'ID_bb1d991f6'].index)
broken_list.append(df.loc[df['ImageId'] == 'ID_c44983aeb'].index)

for idx in broken_list:
    df = df.drop(idx)
print(len(df))
# df = shuffle(df)
df.to_csv("./dataset/train_remove.csv")


# Ignore Mask cleaning

In [ ]:
def CreateMaskImages(dir_path,image_Name):
    train_image = cv2.imread("{}_images/{}.jpg".format(dir_path,image_Name))
    if not os.path.isfile("{}_masks/{}.jpg".format(dir_path,image_Name)):
        return train_image
    image_mask = cv2.imread("{}_masks/{}.jpg".format(dir_path,image_Name),0)
    image_mask = image_mask//200*255
#     image_mask = np.ceil((image_mask//255))*255
#     image_mask = np.array(image_mask,dtype=np.uint8)

#     print(np.max(image_mask),np.min(image_mask))

    image_mask_inv = cv2.bitwise_not(image_mask)
    res = cv2.bitwise_and(train_image,train_image,mask = image_mask_inv)
    #cut upper half,because it doesn't contain cars.
#     res = res[res.shape[0] // 2:]
    return res
    
train_pd = pd.read_csv("./dataset/train_remove.csv")
train_dir = "./dataset/train"
data_len = len(train_pd)

for idx in range(0,data_len):
    if idx%500==0:
        print("idx:",idx)
    file_name = train_pd['ImageId'].iloc[idx]
    res = CreateMaskImages(train_dir,file_name)
    cv2.imwrite("./dataset/masked_train/" + file_name + ".jpg", res)

    
test_pd = pd.read_csv("./dataset/sample_submission.csv")
test_dir = "./dataset/test"
data_len = len(test_pd)

for idx in range(0,data_len):
    if idx%500==0:
        print("idx:",idx)
    file_name = test_pd['ImageId'].iloc[idx]
    res = CreateMaskImages(test_dir,file_name)
    cv2.imwrite("./dataset/masked_test/" + file_name + ".jpg", res)
    

# Image processing

In [ ]:
# IMG_WIDTH = 1024
# IMG_HEIGHT = 320  #IMG_HEIGHT = IMG_WIDTH // 16 * 5

# IMG_WIDTH = 1536
# IMG_HEIGHT = 512
# MODEL_SCALE = 8

# IMG_WIDTH = 2048
# IMG_HEIGHT = IMG_WIDTH // 4

IMG_WIDTH = 512
IMG_HEIGHT = 512
MODEL_SCALE = 8

def imread(path, fast_mode=False):
    img = cv2.imread(path)
    if not fast_mode and img is not None and len(img.shape) == 3:
        img = np.array(img[:, :, ::-1])
    return img

def rotate(x, angle): 
    x = x + angle 
    x = x - (x + np.pi) // (2 * np.pi) * 2 * np.pi 
    return x

def _regr_preprocess(regr_dict, flip=False):
    if flip:
        for k in ['x', 'pitch', 'roll']:
            regr_dict[k] = -regr_dict[k]
    for name in ['x', 'y', 'z']:
        regr_dict[name] = regr_dict[name] / 100
    regr_dict['roll'] = rotate(regr_dict['roll'], np.pi)
    regr_dict['pitch_sin'] = sin(regr_dict['pitch'])
    regr_dict['pitch_cos'] = cos(regr_dict['pitch'])
    regr_dict.pop('pitch')
    regr_dict.pop('id')
    return regr_dict

def _regr_back(regr_dict):
    for name in ['x', 'y', 'z']:
        regr_dict[name] = regr_dict[name] * 100
    regr_dict['roll'] = rotate(regr_dict['roll'], -np.pi)
    
    pitch_sin = regr_dict['pitch_sin'] / np.sqrt(regr_dict['pitch_sin']**2 + regr_dict['pitch_cos']**2)
    pitch_cos = regr_dict['pitch_cos'] / np.sqrt(regr_dict['pitch_sin']**2 + regr_dict['pitch_cos']**2)
    regr_dict['pitch'] = np.arccos(pitch_cos) * np.sign(pitch_sin)
    return regr_dict

def preprocess_image(img, flip=False):
    img = img[img.shape[0] // 2:]
#     bg = np.ones_like(img) * img.mean(1, keepdims=True).astype(img.dtype)
#     bg = bg[:, :img.shape[1] // 6]
#     img = np.concatenate([bg, img, bg], 1)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    if flip:
        img = img[:,::-1]
    return (img / 255).astype('float32')

def get_mask_and_regr(img, labels, flip=False):
    mask = np.zeros([IMG_HEIGHT // MODEL_SCALE, IMG_WIDTH // MODEL_SCALE], dtype='float32')
    regr_names = ['x', 'y', 'z', 'yaw', 'pitch', 'roll']
    regr = np.zeros([IMG_HEIGHT // MODEL_SCALE, IMG_WIDTH // MODEL_SCALE, 7], dtype='float32')
    coords = str2coords(labels)
    xs, ys = get_img_coords(labels)
    for x, y, regr_dict in zip(xs, ys, coords):
        x, y = y, x
        x = (x - img.shape[0] // 2) * IMG_HEIGHT / (img.shape[0] // 2) / MODEL_SCALE
        x = np.round(x).astype('int')
#         y = (y + img.shape[1] // 6) * IMG_WIDTH / (img.shape[1] * 4/3) / MODEL_SCALE
        y = (y) * IMG_WIDTH / (img.shape[1]) / MODEL_SCALE
        y = np.round(y).astype('int')
        if x >= 0 and x < IMG_HEIGHT // MODEL_SCALE and y >= 0 and y < IMG_WIDTH // MODEL_SCALE:
            mask[x, y] = 1
            regr_dict = _regr_preprocess(regr_dict, flip)
            regr[x, y] = [regr_dict[n] for n in sorted(regr_dict)]
    if flip:
        mask = np.array(mask[:,::-1])
        regr = np.array(regr[:,::-1])
    return mask, regr

# mAP calculating

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt, acos, pi, sin, cos
from scipy.spatial.transform import Rotation as R
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from inspect import signature
import matplotlib.pyplot as plt
from multiprocessing import Pool

def expand_df(df, PredictionStringCols):
    df = df.dropna().copy()
    df['NumCars'] = [int((x.count(' ')+1)/7) for x in df['PredictionString']]
    image_id_expanded = [item for item, count in zip(df['ImageId'], df['NumCars']) for i in range(count)]
    prediction_strings_expanded = df['PredictionString'].str.split(' ',expand = True).values.reshape(-1,7).astype(float)
    prediction_strings_expanded = prediction_strings_expanded[~np.isnan(prediction_strings_expanded).all(axis=1)]
    df = pd.DataFrame(
        {
            'ImageId': image_id_expanded,
            PredictionStringCols[0]:prediction_strings_expanded[:,0],
            PredictionStringCols[1]:prediction_strings_expanded[:,1],
            PredictionStringCols[2]:prediction_strings_expanded[:,2],
            PredictionStringCols[3]:prediction_strings_expanded[:,3],
            PredictionStringCols[4]:prediction_strings_expanded[:,4],
            PredictionStringCols[5]:prediction_strings_expanded[:,5],
            PredictionStringCols[6]:prediction_strings_expanded[:,6]
        })
    return df

def str2coords(s, names):
    coords = []
    for l in np.array(s.split()).reshape([-1, 7]):
        coords.append(dict(zip(names, l.astype('float'))))
    return coords

def TranslationDistance(p,g, abs_dist = False):
    dx = p['x'] - g['x']
    dy = p['y'] - g['y']
    dz = p['z'] - g['z']
    diff0 = (g['x']**2 + g['y']**2 + g['z']**2)**0.5
    diff1 = (dx**2 + dy**2 + dz**2)**0.5
    if abs_dist:
        diff = diff1
    else:
        diff = diff1/diff0
    return diff

def RotationDistance(p, g):
    true=[ g['pitch'] ,g['yaw'] ,g['roll'] ]
    pred=[ p['pitch'] ,p['yaw'] ,p['roll'] ]
    q1 = R.from_euler('xyz', true)
    q2 = R.from_euler('xyz', pred)
    diff = R.inv(q2) * q1
    W = np.clip(diff.as_quat()[-1], -1., 1.)
    
    # in the official metrics code:
    # https://www.kaggle.com/c/pku-autonomous-driving/overview/evaluation
    #   return Object3D.RadianToDegree( Math.Acos(diff.W) )
    # this code treat θ and θ+2π differntly.
    # So this should be fixed as follows.
    W = (acos(W)*360)/pi
    if W > 180:
        W = 360 - W
    return W

def print_pr_curve(result_flg, scores, recall_total=1):
    average_precision = average_precision_score(result_flg, scores)
    precision, recall, _ = precision_recall_curve(result_flg, scores)
    recall *= recall_total
    plt.step(recall, precision, color='b', alpha=0.2, where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.show()
    
thres_tr_list = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
thres_ro_list = [50, 45, 40, 35, 30, 25, 20, 15, 10, 5]

def check_match(idx,train_df,valid_df):
    keep_gt=False
    thre_tr_dist = thres_tr_list[idx]
    thre_ro_dist = thres_ro_list[idx]
    train_dict = {imgID:str2coords(s, names=['carid_or_score', 'pitch', 'yaw', 'roll', 'x', 'y', 'z']) for imgID,s in zip(train_df['ImageId'],train_df['PredictionString'])}
    valid_dict = {imgID:str2coords(s, names=['pitch', 'yaw', 'roll', 'x', 'y', 'z', 'carid_or_score']) for imgID,s in zip(valid_df['ImageId'],valid_df['PredictionString'])}
    result_flg = [] # 1 for TP, 0 for FP
    scores = []
    MAX_VAL = 10**10
    for img_id in valid_dict:
        for pcar in sorted(valid_dict[img_id], key=lambda x: -x['carid_or_score']):
            # find nearest GT
            min_tr_dist = MAX_VAL
            min_idx = -1
            for idx, gcar in enumerate(train_dict[img_id]):
                tr_dist = TranslationDistance(pcar,gcar)
                if tr_dist < min_tr_dist:
                    min_tr_dist = tr_dist
                    min_ro_dist = RotationDistance(pcar,gcar)
                    min_idx = idx
                    
            # set the result
            if min_tr_dist < thre_tr_dist and min_ro_dist < thre_ro_dist:
                if not keep_gt:
                    train_dict[img_id].pop(min_idx)
                result_flg.append(1)
            else:
                result_flg.append(0)
            scores.append(pcar['carid_or_score'])
    
    return result_flg, scores    
    

In [3]:
max_workers = 10
def calculate_mAp(val_file_name):
    valid_df = pd.read_csv(val_file_name)
    expanded_valid_df = expand_df(valid_df, ['pitch','yaw','roll','x','y','z','Score'])
    valid_df = valid_df.fillna('')
    train_df = pd.read_csv('./dataset/train_remove.csv')    
    train_df = train_df[train_df.ImageId.isin(valid_df.ImageId.unique())]
    expanded_train_df = expand_df(train_df, ['model_type','pitch','yaw','roll','x','y','z'])

    n_gt = len(expanded_train_df)
    ap_list = []
    p = Pool(processes=max_workers)
    
    input_data = []
    for i in range(10):
        input_data.append((i,train_df,valid_df))
    
    for result_flg, scores in p.starmap(check_match,input_data):
#     for result_flg, scores in p.imap(check_match, input_data):
        if np.sum(result_flg) > 0:
            n_tp = np.sum(result_flg)
            recall = n_tp/n_gt
            scores = np.random.rand(len(result_flg))
            ap = average_precision_score(result_flg, scores)*recall
    #         print_pr_curve(result_flg, scores, recall)
        else:
            ap = 0
        ap_list.append(ap)
    mAp = np.mean(ap_list)
    return mAp


In [ ]:
# valid_df = pd.read_csv("./best_model_kgroup/val_k20_model5_rmsprop_5e-4_b2_effb5_1536x320_flip0.5_bgTrue_peakScalar1_Clip1e-12_multiloss_Ep55_loss2.8832.csv")
# expanded_valid_df = expand_df(valid_df, ['pitch','yaw','roll','x','y','z','Score'])
# valid_df = valid_df.fillna('')
# train_df = pd.read_csv('./dataset/train_remove.csv')
# train_df = train_df[train_df.ImageId.isin(valid_df.ImageId.unique())]
# expanded_train_df = expand_df(train_df, ['model_type','pitch','yaw','roll','x','y','z'])
# max_workers = 10
# n_gt = len(expanded_train_df)
# ap_list = []
# p = Pool(processes=10)
# input_data = []

# for i in range(10):
#     input_data.append((i,train_df,valid_df))
    
# for result_flg, scores in p.starmap(check_match,input_data):
# # for result_flg, scores in p.imap(check_match, range(10)):
#     if np.sum(result_flg) > 0:
#         n_tp = np.sum(result_flg)
#         recall = n_tp/n_gt
        
#         ###Get into it later
#         score = np.random.rand(len(result_flg))
#         ap = average_precision_score(result_flg, scores)*recall
#         print_pr_curve(result_flg, scores, recall)
#     else:
#         ap = 0
#     ap_list.append(ap)
# map = np.mean(ap_list)
# print('map:', map)

# val_root = './best_model_pred_0116/No_optimizeXYZ'
# val_root = './best_model_pred_0116/optimizer_para0.1_0.2'
# val_root = './best_model_pred_0116/current_model_distances'
val_root = './best_model_pred_0116/tmp'

for file_name in os.listdir(val_root):
    if file_name.find('val')!=-1:
        continue
    mAp = calculate_mAp("{}/{}".format(val_root,file_name))
    print("File:{}, mAp:{}".format(file_name,mAp))


# Visualize image

In [ ]:
global_data = pd.read_csv("./dataset/critic01_20k.csv")

In [ ]:
for idx in range(0,100):

    i = idx
    img = global_data.iloc[i, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
    img = Image.fromarray(img)
    label = global_data.iloc[i, 0]
    
    if i%10==0:
        plt.pause(.1)
        fig, axes = plt.subplots(1,10,figsize=(16,2))
    axes[i%10].imshow(img,cmap="gray")
    
    #     img = trans(img).cpu().numpy().reshape(28,28)
    #     img = transforms.functional.adjust_brightness(img, 0.01)
    #     img = transforms.functional.adjust_contrast(img, 0.01)
    #     img = transforms.functional.adjust_saturation(img, 5)
#         axes[j][1].imshow(img,cmap="gray")
    
#     img = global_data_dig.iloc[i, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
#     img = Image.fromarray(img)
#     label = global_data_dig.iloc[i, 0]
#     axes[2].imshow(img,cmap="gray")
#     img = trans(img).cpu().numpy().reshape(28,28)
#     axes[3].imshow(img,cmap="gray")
    

    print("Label:",label)

# data.head(5)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

###Confusion matrix
print(classification_report(labels, result,digits=4))
plt.figure(figsize=(10,10))
confusion_mat = confusion_matrix(labels, result)
sn.heatmap(confusion_mat, annot=True, cmap='YlGnBu',fmt="d",linewidths=.5, linecolor='w')
plt.title('Confusion matrix - DIG-MNIST dataset')
plt.ylabel('True label')
plt.xlabel('Predicted label')


In [ ]:
# indices1 = np.where(result!=labels)[0]       #(num)
# indices2 = np.where(result_dig!=labels)[0]
# indices3 = np.where((result!=labels)&(result_dig==labels))[0]
indices4 = np.where((result[:,0]==labels))[0]
indices5 = np.where((result[:,0]==labels)|(result[:,1]==labels))[0]
indices6 = np.where((result[:,0]==labels)|(result[:,1]==labels)|(result[:,2]==labels))[0]

cirtic_idx_list_ans0 = []
idx_list_ans1 = []

cirtic_idx_list_ans9 = []
label_list = []
count = 0

data_num = len(labels)
print("top1 acc:",len(indices4)/data_num)
print("top2 acc:",len(indices5)/data_num)
print("top3 acc:",len(indices6)/data_num)
stop

for i in range(0,len(indices1)):
    idx = indices1[i]
#     img1 = global_dig.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
#     img1 = global_pseudo_data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
    img1 = global_data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))  #value: 0~255
    img1 = Image.fromarray(img1)
    label1 = result[idx]
#     label2 = result_dig[idx]
    label2 = None
    label = labels[idx]
  

#     if label == 0:
#         fig, axes = plt.subplots(1,1,figsize=(2,2))
# #         cirtic_idx_list_ans0.append(idx)
#         axes.imshow(img1,cmap="gray")
#         print(idx)
#         print("Model:",label1," Model2:",label2," Label:",label)
#         plt.pause(.1)
#     else:
#         continue
        

# print(idx_list_ans1)
# np.save("idx_ans1",idx_list_ans1)

# print(cirtic_idx_list_ans0)
# print(cirtic_idx_list_ans9)

cirtic_idx_list_ans0_v2 = [5520,6150,12280,18560,32730]
np.save("critic_idx_ans0_v2",cirtic_idx_list_ans0_v2)

# plt.figure(figsize=(8,4))
# data = np.uint8(cirtic_label_list)
# print(len(data))
# plt.hist(data ,density=0,align="mid",bins=10,rwidth=1)
# plt.xticks(range(10))
# plt.ylabel('frequency')

# plt.figure(figsize=(8,4))
# data = np.uint8(label_list)
# print(len(data))
# plt.hist(data ,density=0,align="mid",bins=10,rwidth=1)
# plt.xticks(range(10))
# plt.ylabel('frequency')


# 31 1 0 59969